## Script to pull census data.

This notebook includes a simple script that will pull the requested variables form the 5-year American Community Survey at either the census tract or congressional district level.  The user must request a census api key and enter it in the line that says "my_api_key".  The user should then put the variable names from the ACS5 in the indicated line, seperated by commas.  The user may optionally put names in the indicated line, also seperated by commas.  Both should be surrounded by quotes (single/double quotes are both fine).  Then run the cells of this notebook

This is just a simple application of the library here: https://github.com/datamade/census

If you are missing the required libraries, put these in a new cell and run it:

!pip install census

!pip install us

!pip install tqdm

In [1]:
from census import Census
from us import states
import requests
import pandas as pd
from tqdm import tqdm_notebook
from IPython.display import clear_output

my_api_key = "b141795955348002155a81415a728a61faab9746"# Put your api key here

try:
    census_query_object = Census(my_api_key)
except:
    print("Did you enter an api key surrounded by quotes?")
    


variables_of_interest = ['NAME', 'B19013_001E', 'B19013B_001E']
custom_names = ['default', 'default', 'default']


def grab_by_tracts(variables_list, names=custom_names, file_name = "tracts_data.csv"):
    """This gathers the requested variables from the 5-year American Community 
    Survey at the census tract level.  Note, this will take longer then the 
    district data, perhaps a couple minutes."""

    stats = []
    census_query_object = Census(my_api_key)
    for x in tqdm_notebook(states.STATES):
        stats = stats + census_query_object.acs5.state_county_tract(
            tuple(variables_list), x.fips, Census.ALL, Census.ALL)
    df = pd.DataFrame(stats)
    df['State Name'] = df['state'].apply(lambda x: states.STATES_AND_TERRITORIES[int(x)-1].abbr)
    
    if names and type(names)==list:
        for i in range( min(len(names), df.shape[1]) ):
            if names[i]!="default" and names[i]!='':
                df.columns[i]=names[i]
            
    df.to_csv(file_name)
    clear_output()
    display(df.head())
#     return df
    
def grab_by_districts(variables_list, names=custom_names, file_name = "district_data.csv"):
    """This gathers the requested variables from the 5-year American Community 
    Survey at the congressional district level."""
    stats = []
    census_query_object = Census(my_api_key)
    for x in tqdm_notebook(states.STATES, leave=False):
        stats = stats + census_query_object.acs5.state_district(
            tuple(variables_list), x.fips , Census.ALL)
    df = pd.DataFrame(stats)
    df['State Name'] = df['state'].apply(lambda x: states.STATES_AND_TERRITORIES[int(x)-1].abbr)
        
    if names and type(names)==list:
        for i in range( min(len(names), df.shape[1]) ):
            if names[i]!="default" and names[i]!='':
                df.columns[i]=names[i]
            
    df.to_csv(file_name)
    clear_output()
    display(df.head())
#     return df

In [2]:
"""Run this cell to save district data."""

"""You may optionally indicate new variables and names by removing the hashmarks in front of the next two lines"""
# variables_of_interest = ('NAME', 'B19013_001E', 'B19013B_001E')
# custom_names = ('default', 'default', 'default')


df = grab_by_districts(variables_of_interest)

,B19013B_001E,B19013_001E,NAME,congressional district,state,State Name
0,26956.0,44824.0,"Congressional District 1 (115th Congress), Ala...",01,01,AL
1,29348.0,43173.0,"Congressional District 2 (115th Congress), Ala...",02,01,AL
2,30393.0,42953.0,"Congressional District 3 (115th Congress), Ala...",03,01,AL
3,28789.0,40267.0,"Congressional District 4 (115th Congress), Ala...",04,01,AL
4,35509.0,51830.0,"Congressional District 5 (115th Congress), Ala...",05,01,AL


In [3]:
"""Run this cell to save census tract data.  Note, this will take longer then the district data, perhaps a couple minutes."""

"""You may optionally indicate new variables and names by removing the hashmarks in front of the next two lines"""
# variables_of_interest = ('NAME', 'B19013_001E', 'B19013B_001E')
# custom_names = ('default', 'default', 'default')

grab_by_tracts(variables_of_interest)

,B19013B_001E,B19013_001E,NAME,county,state,tract,State Name
0,26157.0,66000.0,"Census Tract 201, Autauga County, Alabama",001,01,020100,AL
1,30052.0,41107.0,"Census Tract 202, Autauga County, Alabama",001,01,020200,AL
2,42554.0,51250.0,"Census Tract 203, Autauga County, Alabama",001,01,020300,AL
3,39808.0,52704.0,"Census Tract 204, Autauga County, Alabama",001,01,020400,AL
4,41023.0,52463.0,"Census Tract 205, Autauga County, Alabama",001,01,020500,AL
